# Natural Language Processing - US Airline Twitter Sentiment Analysis

## Importing the libraries

In [220]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

## Importing the dataset

In [221]:
dataset = pd.read_csv('Tweets_usairline.csv')

## Cleaning the texts

In [222]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
corpus = []
n = len(dataset['text'])

for i in range(0, n):
    review = re.sub('[^a-zA-Z]', ' ', dataset['text'][i])
    review = review.lower()
    review = review.split()
    lemm = WordNetLemmatizer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [lemm.lemmatize(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\savit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Creating the Bag of Words model

In [223]:
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()
y = dataset["airline_sentiment"].values

## Splitting the dataset into the Training set and Test set

In [224]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

## Training the Naive Bayes model on the Training set

In [225]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

## Predicting the Test set results

In [226]:
y_pred = classifier.predict(X_test)
print(y_pred,y_test)

['negative' 'negative' 'negative' ... 'negative' 'positive' 'negative'] ['negative' 'negative' 'negative' ... 'negative' 'negative' 'negative']


## Making the Confusion Matrix

In [227]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[873 372 625]
 [120 210 284]
 [ 81  58 305]]


0.47404371584699456

The accuracy is 47% with naive bayes model which is very low. Lets try to predict with some reviews.

## Predicting if a single review is positive or neutral or negative

### Positive review

Use our model to predict if the following review:

"American airlines is the best airline in the US"

is positive or negative or neutral.

In [228]:
new_review = 'American airlines is the best airline in the US'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
lemm = WordNetLemmatizer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [lemm.lemmatize(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_X_test)
print(new_y_pred)

['positive']


The review was correctly predicted as positive by our model.

### Neutral review

Use our model to predict if the following review:

"the southwest airlines fly to dallas everyday"

is positive or negative or neutral.

In [229]:
new_review = 'the southwest airlines fly to dallas everyday'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
lemm = WordNetLemmatizer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [lemm.lemmatize(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_X_test)
print(new_y_pred)

['positive']


The review was predicted incorrectly as positive instead of neutral.

### Negative review

Use our model to predict if the following review:

"I never fly in american airlines since its not good"

is positive or negative or neutral.

In [230]:
new_review = 'I never fly in american airlines since its not good'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
lemm = WordNetLemmatizer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [lemm.lemmatize(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_X_test)
print(new_y_pred)

['positive']


The review was predicted incorrectly as positive instead of negative.

## Implementing the same using Deep Learning ANN model

### Performing one hot coding for the target variable

In [203]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)
print(dummy_y)

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


### Splitting the train and test sets

In [204]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size = 0.20, random_state = 0)

### Initialize the ANN model

In [205]:
annNLP = tf.keras.models.Sequential()

### Add the first input layer with input dimensions as 12573 and input neurons = 200

In [99]:
annNLP.add(tf.keras.layers.Dense(units=200, input_dim = 12573, activation='relu'))

### Add the second hidden layer

In [207]:
annNLP.add(tf.keras.layers.Dense(units=200, activation='relu'))

## Add the output layer

In [208]:
annNLP.add(tf.keras.layers.Dense(units=3, activation='softmax'))

## Compile the ANN model

In [210]:
annNLP.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Train the ANN model

In [211]:
annNLP.fit(X_train, y_train, batch_size = 32, epochs = 100)

Train on 11712 samples
Epoch 1/100
11712/11712 [==============================] - 6s 534us/sample - loss: 0.6042 - accuracy: 0.7511
Epoch 2/100
11712/11712 [==============================] - 6s 495us/sample - loss: 0.3268 - accuracy: 0.8751
Epoch 3/100
11712/11712 [==============================] - 6s 479us/sample - loss: 0.2013 - accuracy: 0.9301
Epoch 4/100
11712/11712 [==============================] - 6s 485us/sample - loss: 0.1297 - accuracy: 0.9559
Epoch 5/100
11712/11712 [==============================] - 6s 488us/sample - loss: 0.0893 - accuracy: 0.9728
Epoch 6/100
11712/11712 [==============================] - 6s 517us/sample - loss: 0.0656 - accuracy: 0.9804
Epoch 7/100
11712/11712 [==============================] - 6s 498us/sample - loss: 0.0509 - accuracy: 0.9851
Epoch 8/100
11712/11712 [==============================] - 6s 498us/sample - loss: 0.0423 - accuracy: 0.9886 - los
Epoch 9/100
11712/11712 [==============================] - 6s 499us/sample - loss: 0.0357 - accurac

11712/11712 [==============================] - 6s 507us/sample - loss: 0.0106 - accuracy: 0.9946
Epoch 75/100
11712/11712 [==============================] - 6s 512us/sample - loss: 0.0107 - accuracy: 0.9951
Epoch 76/100
11712/11712 [==============================] - 6s 541us/sample - loss: 0.0106 - accuracy: 0.9948
Epoch 77/100
11712/11712 [==============================] - 6s 498us/sample - loss: 0.0101 - accuracy: 0.9956
Epoch 78/100
11712/11712 [==============================] - 6s 496us/sample - loss: 0.0105 - accuracy: 0.9948 - loss: 0.0104 - 
Epoch 79/100
11712/11712 [==============================] - 6s 509us/sample - loss: 0.0106 - accuracy: 0.9948
Epoch 80/100
11712/11712 [==============================] - 6s 505us/sample - loss: 0.0107 - accuracy: 0.9949
Epoch 81/100
11712/11712 [==============================] - 6s 499us/sample - loss: 0.0102 - accuracy: 0.9950
Epoch 82/100
11712/11712 [==============================] - 6s 504us/sample - loss: 0.0104 - accuracy: 0.9948
Epoch

## Converting the predictions in test set to 1 D array

In [212]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = annNLP.predict(X_test)

y_pred = y_pred.round()


y_pred_1d = []
y_test_1d = []

for x in y_pred:
    if x[0] == 1:
        y_pred_1d.append("negative")
    #elif x[1] == 1:
    #    y_pred_1d.append("neutral")
    elif x[2] == 1:
        y_pred_1d.append("positive")
    else:
        y_pred_1d.append("neutral")

for x in y_test:
    if x[0] == 1:
        y_test_1d.append("negative")
    #elif x[1] == 1:
    #    y_test_1d.append("neutral")
    elif x[2] == 1:
        y_test_1d.append("positive")
    else:
        y_test_1d.append("neutral")        

cm = confusion_matrix(y_test_1d, y_pred_1d)
print(cm)
x = accuracy_score(y_test_1d, y_pred_1d,normalize = False)
print("Accuracy = ",x/len(y_pred_1d))

[[1570  224   76]
 [ 172  363   79]
 [  73   90  281]]
Accuracy =  0.7561475409836066


The accuracy is 76% with naive bayes model which is better than naive bayes model. Lets try to predict with some reviews.

## Predicting if a single review is positive or neutral or negative

### Positive review

Use our ANN model to predict if the following review:

"American airlines is the best airline in the US"

is positive or negative or neutral.

In [215]:
new_review = 'American airlines is the best airline in the US'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
lemm = WordNetLemmatizer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [lemm.lemmatize(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = annNLP.predict(new_X_test)
x  = new_y_pred.round()

if x[0][0] == 1:
    print("The sentiment of the review is Negative")
elif x[0][2] == 1:
    print("The sentiment of the review is Positive")
else:
    print("The sentiment of the review is Neutral")

The sentiment of the review is Positive


The review was correctly predicted as positive by ANN model.

### Neutral review

Use our model to predict if the following review:

"the southwest airlines fly to dallas everyday"

is positive or negative or neutral.

In [217]:
new_review = 'the southwest airlines fly to dallas everyday'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
lemm = WordNetLemmatizer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [lemm.lemmatize(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = annNLP.predict(new_X_test)
x  = new_y_pred.round()

if x[0][0] == 1:
    print("The sentiment of the review is Negative")
elif x[0][2] == 1:
    print("The sentiment of the review is Positive")
else:
    print("The sentiment of the review is Neutral")

The sentiment of the review is Neutral


The review was correctly predicted as neutral by ANN model.

### Negative review

Use our model to predict if the following review:

"I never fly in american airlines since its not good"

is positive or negative or neutral.

In [219]:
new_review = 'I never fly in american airlines since its not good'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
lemm = WordNetLemmatizer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [lemm.lemmatize(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = annNLP.predict(new_X_test)
x  = new_y_pred.round()

if x[0][0] == 1:
    print("The sentiment of the review is Negative")
elif x[0][2] == 1:
    print("The sentiment of the review is Positive")
else:
    print("The sentiment of the review is Neutral")

The sentiment of the review is Negative


The review was correctly predicted as negative by ANN model.

## Conclusion

### Overall the ANN model gives a better prediction on the sentiment rather than the naive bayes model